In [ ]:
import pandas
import matplotlib.pyplot as plt
from collections import Counter
import string
from collections import Counter
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras import losses

wf = pandas.read_csv("/home/yash/Code/PresidentialSpeechClassification/EMPOLITICON.csv")
wf

2025-05-21 12:20:15.223292: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-21 12:20:15.518857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-21 12:20:15.662737: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-21 12:20:15.696253: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 12:20:15.883672: I tensorflow/core/platform/cpu_feature_guar

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,Russia,16/07/2021,Vladimir Putin,Meeting of APEC Economic Leaders,"Madam Chair,\n\nColleagues,\n\nFirst of all, ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,OPTIMISM,DEVELOPMENT
1,Russia,2021-09-05 00:00:00,Vladimir Putin,Victory Parade on Red Square,"Citizens of Russia,\n\nDear veterans,\n\nComra...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,JOY,NATIONALISM
2,Russia,2021-08-04 00:00:00,Vladimir Putin,Meeting on the results of implementing Preside...,"Good afternoon, colleagues.\n\nLet’s start.\n\...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
3,Russia,21-11-2020,Vladimir Putin,G20 Summit,"Colleagues,\n\nThe scope of problems humanity ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
4,Russia,20-11-2020,Vladimir Putin,Address to participants in Nuremberg Lessons f...,"Colleagues, friends,\n\nFirst of all, I would ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,UPSET,EXTREMISM
...,...,...,...,...,...,...,...,...,...,...
2005,USA,05/02/2013,Barack Obama,\nRemarks by the President,"Good afternoon, everybody. \n\nI wanted to sa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,DEVELOPMENT
2006,USA,04/02/2013,Barack Obama,\nRemarks by the President on Preventing Gun V...,"Hello, everybody. Please have a seat. Have a...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,OTHERS
2007,USA,02/02/2013,Barack Obama,\nWeekly Address: A Balanced Approach to Growi...,"Hi, everybody. \n\nIn the coming weeks, we fa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,OPTIMISM,DEVELOPMENT
2008,USA,28/01/2013,Barack Obama,\nRemarks by the President Before Meeting with...,"Well, Vice President Biden and I just want to...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,OPTIMISM,OTHERS


In [2]:
def count_unique_word(wf):
    count = Counter()
    for word in wf['Emotion']:
        count[word] += 1
    return count

counter = count_unique_word(wf)

In [3]:
print(counter.most_common())

[('NEUTRAL', 589), ('OPTIMISM', 583), ('JOY', 525), ('UPSET', 313)]


In [4]:
#labeling the emotion column such that the most common emotion is 0, the second most common is 1 and so on
for i in range(len(wf)):
    if wf['Emotion'][i] == counter.most_common()[0][0]:
        wf['Emotion'][i] = 0
    elif wf['Emotion'][i] == counter.most_common()[1][0]:
        wf['Emotion'][i] = 1
    elif wf['Emotion'][i] == counter.most_common()[2][0]:
        wf['Emotion'][i] = 2
    elif wf['Emotion'][i] == counter.most_common()[3][0]:
        wf['Emotion'][i] = 3

#these labels will be used to train the model

/tmp/ipykernel_43649/1047653752.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Emotion'][i] = 1
/tmp/ipykernel_43649/1047653752.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values

In [5]:
wf

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,Russia,16/07/2021,Vladimir Putin,Meeting of APEC Economic Leaders,"Madam Chair,\n\nColleagues,\n\nFirst of all, ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,1,DEVELOPMENT
1,Russia,2021-09-05 00:00:00,Vladimir Putin,Victory Parade on Red Square,"Citizens of Russia,\n\nDear veterans,\n\nComra...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,2,NATIONALISM
2,Russia,2021-08-04 00:00:00,Vladimir Putin,Meeting on the results of implementing Preside...,"Good afternoon, colleagues.\n\nLet’s start.\n\...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT
3,Russia,21-11-2020,Vladimir Putin,G20 Summit,"Colleagues,\n\nThe scope of problems humanity ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT
4,Russia,20-11-2020,Vladimir Putin,Address to participants in Nuremberg Lessons f...,"Colleagues, friends,\n\nFirst of all, I would ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,3,EXTREMISM
...,...,...,...,...,...,...,...,...,...,...
2005,USA,05/02/2013,Barack Obama,\nRemarks by the President,"Good afternoon, everybody. \n\nI wanted to sa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,DEVELOPMENT
2006,USA,04/02/2013,Barack Obama,\nRemarks by the President on Preventing Gun V...,"Hello, everybody. Please have a seat. Have a...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,OTHERS
2007,USA,02/02/2013,Barack Obama,\nWeekly Address: A Balanced Approach to Growi...,"Hi, everybody. \n\nIn the coming weeks, we fa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,1,DEVELOPMENT
2008,USA,28/01/2013,Barack Obama,\nRemarks by the President Before Meeting with...,"Well, Vice President Biden and I just want to...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,1,OTHERS


In [6]:
# wf = wf.head(739)

In [7]:
wf.iloc[76]['Text_of_Speech']

'Members of the State Duma, friends,\n\nIt gives me great pleasure to welcome you and congratulate you on the opening of the seventh convocation of the State Duma.\n\nToday in this hall, we have representatives of all the 85 Russian regions, including those elected in Crimea and Sevastopol.\n\nOur citizens have entrusted you with the distinguished title of a Parliament Member. The election campaign was transparent and the competition was honest. The results demonstrate the will and choice of Russian society. I see a guarantee of the legislative branch’s strength and authority in the absolute legitimacy of the parliament.\n\nThe role of the State Duma as a representative body has significantly increased. Half of the mandates were given to candidates directly elected by residents of specific areas, districts and regions.\n\nOverall, the composition of the State Duma has changed markedly. Representatives of public, professional and local organisations have joined it. I believe that as a r

In [8]:
## one thing we can do is to use the model from the presidential classifier and then, if a given speech is classified
## as angry or some other negative mood, and it includes the word "US, NATO", or something related to the west, we classify it as adversarial

In [9]:
#removing the punctuation
def remove_punctuation(speech):
    translator = str.maketrans("", "", string.punctuation)
    return speech.translate(translator)

In [10]:
#removing the stop words (commonly used words that do not add much meaning to a sentence)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    #removes stop words
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)


[nltk_data] Downloading package stopwords to /home/yash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#drop rows without string in speech
for i in range(len(wf) - 1, -1, -1):
        if not (isinstance(wf.iloc[i]['Text_of_Speech'], str)):
                wf = wf.drop(i)

In [12]:
#counting the number of unique words in each speech


def count_unique_word(wf_training):
    count = Counter()
    for i in range(len(wf_training)):
        for word in wf_training.iloc[i]['Text_of_Speech'].split():
            count[word] += 1
    return count

counter = count_unique_word(wf)

In [13]:
counter

Counter({'the': 134604,
         'and': 96168,
         'to': 85823,
         'of': 78755,
         'in': 46581,
         'a': 42816,
         'that': 33939,
         'is': 28478,
         'we': 26510,
         'for': 26221,
         'our': 23604,
         'I': 22167,
         'have': 17936,
         'are': 17502,
         'this': 16576,
         'on': 16571,
         'will': 15741,
         'with': 15699,
         'be': 13696,
         'And': 13589,
         'as': 13231,
         'We': 11975,
         'you': 11408,
         'it': 11100,
         'all': 10816,
         'not': 10565,
         'has': 8993,
         'their': 8943,
         'The': 8942,
         'more': 8673,
         'by': 8321,
         '—': 8182,
         'from': 7947,
         'people': 7455,
         'at': 7335,
         'an': 7097,
         'who': 7002,
         'they': 6790,
         'can': 6710,
         'new': 5860,
         'was': 5826,
         'but': 5470,
         'China': 5420,
         'been': 5363,
        

In [14]:
num_unique_words = len(counter)

In [15]:
print(len(wf))

2010


In [16]:
wf_training = wf[::2]
wf_testing = wf[1::2]

In [17]:
wf_testing.reset_index(inplace=True)
wf_testing.drop('index', axis=1, inplace=True)
wf_testing

/tmp/ipykernel_43649/3894747268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_testing.drop('index', axis=1, inplace=True)


,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,Russia,2021-09-05 00:00:00,Vladimir Putin,Victory Parade on Red Square,"Citizens of Russia,\n\nDear veterans,\n\nComra...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,2,NATIONALISM
1,Russia,21-11-2020,Vladimir Putin,G20 Summit,"Colleagues,\n\nThe scope of problems humanity ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT
2,Russia,16-11-2020,Vladimir Putin,Security Council meeting,"Colleagues, good afternoon.\n\nToday we will d...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT
3,Russia,26-10-2020,Vladimir Putin,Statement by Vladimir Putin on additional step...,The Russian Federation continues to believe th...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,3,INTERNATIONAL AFFAIRS
4,Russia,22-09-2020,Vladimir Putin,75th session of the UN General Assembly,"Mr. President, Mr. Secretary-General, colleagu...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,INTERNATIONAL AFFAIRS
...,...,...,...,...,...,...,...,...,...,...
1000,USA,19/02/2013,Barack Obama,\nRemarks by the President on the Sequester,"Good morning, everybody. (Applause.) Please ...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,3,DEVELOPMENT
1001,USA,14/02/2013,Barack Obama,\nRemarks by the President on Early Childhood ...,"Hello, everybody! (Applause.) Well, it is gr...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,2,DEVELOPMENT
1002,USA,05/02/2013,Barack Obama,\nRemarks by the President,"Good afternoon, everybody. \n\nI wanted to sa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,DEVELOPMENT
1003,USA,02/02/2013,Barack Obama,\nWeekly Address: A Balanced Approach to Growi...,"Hi, everybody. \n\nIn the coming weeks, we fa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,1,DEVELOPMENT


In [18]:
wf_training.reset_index(inplace=True)
wf_training.drop('index', axis=1, inplace=True)
wf_training

/tmp/ipykernel_43649/1694703126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_training.drop('index', axis=1, inplace=True)


,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,Russia,16/07/2021,Vladimir Putin,Meeting of APEC Economic Leaders,"Madam Chair,\n\nColleagues,\n\nFirst of all, ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,1,DEVELOPMENT
1,Russia,2021-08-04 00:00:00,Vladimir Putin,Meeting on the results of implementing Preside...,"Good afternoon, colleagues.\n\nLet’s start.\n\...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT
2,Russia,20-11-2020,Vladimir Putin,Address to participants in Nuremberg Lessons f...,"Colleagues, friends,\n\nFirst of all, I would ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,3,EXTREMISM
3,Russia,2020-10-11 00:00:00,Vladimir Putin,Statement by the President of Russia,"Today, on November 9, President of the Republi...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,INTERNATIONAL AFFAIRS
4,Russia,25-09-2020,Vladimir Putin,Statement by President of Russia Vladimir Puti...,One of today’s major strategic challenges is t...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,1,INTERNATIONAL AFFAIRS
...,...,...,...,...,...,...,...,...,...,...
1000,USA,26/02/2013,Barack Obama,\nRemarks by the President on the Impact of th...,"Hello, Newport News! (Applause.) Well, it is...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,1,DEVELOPMENT
1001,USA,16/02/2013,Barack Obama,\nWeekly Address: Following the President’s Pl...,"Hi, everybody. This week, I’ve been travelin...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,1,DEVELOPMENT
1002,USA,09/02/2013,Barack Obama,\nWeekly Address: Averting the Sequester and F...,"\n\nHi, everybody. Over the last few years, De...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,DEVELOPMENT
1003,USA,04/02/2013,Barack Obama,\nRemarks by the President on Preventing Gun V...,"Hello, everybody. Please have a seat. Have a...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,OTHERS


In [19]:
training_sentences = wf_training.Text_of_Speech.to_numpy()
#training labels
training_labels = wf_training.Emotion.to_numpy()

validation_sentences = wf_testing.Text_of_Speech.to_numpy()
#training labels
validation_labels = wf_testing.Emotion.to_numpy()

In [20]:
training_sentences.shape, validation_sentences.shape

((1005,), (1005,))

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(training_sentences)

In [22]:
word_index = tokenizer.word_index

In [23]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

In [24]:
#padding sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

#max number of words per sequence
max_length = 100

training_padded = pad_sequences(training_sequences, maxlen=max_length, padding="post", truncating="post")
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding="post", truncating="post")
training_padded.shape, validation_padded.shape

((1005, 100), (1005, 100))

In [25]:
reverse_word_index_dictionary = dict([(idx, word) for (word, idx) in word_index.items()])

In [26]:
def decode(sequence):
    #takes the sequence and decodes it according to the reverse dictionary
    return " ".join([reverse_word_index_dictionary.get(idx, "?") for idx in sequence])

In [ ]:
#LSTM Model
import tensorflow.keras as keras


model = keras.models.Sequential()

#input layer
#embedding layer converts the words into vectors of fixed size
model.add(layers.Embedding(num_unique_words, 32))
#dropout rate drops out a certain percentage of input data so that the
#model learns more robust ways of learning and does not overfit
model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

I0000 00:00:1747844423.821717   43649 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1747844424.116952   43649 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1747844424.117024   43649 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1747844424.122843   43649 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1747844424.122939   43649 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
loss = 'sparse_categorical_crossentropy'
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [29]:
print(training_padded.dtype, training_padded.shape)

int32 (1005, 100)


In [30]:

training_labels = np.array(training_labels).astype(np.int64)
validation_labels = np.array(validation_labels).astype(np.int64)

In [31]:
print(training_padded.dtype, training_padded.shape)
print(training_labels.dtype, training_labels.shape)

int32 (1005, 100)
int64 (1005,)


In [32]:
print(training_padded)

[[3268 2808  325 ...   40   85  325]
 [ 109  813  325 ... 3895 2382 8777]
 [ 325  224   95 ... 1811    2  482]
 ...
 [1798  258   92 ...   25  901   99]
 [2144  258  756 ...   58   75    3]
 [ 118 1185  128 ...  172    8  734]]


In [ ]:
model.fit(training_padded, training_labels, epochs=10, validation_data=(validation_padded, validation_labels), verbose=2)

Epoch 1/20
32/32 - 33s - 1s/step - accuracy: 0.2806 - loss: 1.4718 - val_accuracy: 0.2836 - val_loss: 1.3958
Epoch 2/20
32/32 - 27s - 850ms/step - accuracy: 0.2756 - loss: 1.3625 - val_accuracy: 0.2995 - val_loss: 1.3654
Epoch 3/20
32/32 - 27s - 831ms/step - accuracy: 0.2915 - loss: 1.3410 - val_accuracy: 0.2488 - val_loss: 1.3745
Epoch 4/20
32/32 - 28s - 866ms/step - accuracy: 0.3483 - loss: 1.3014 - val_accuracy: 0.3005 - val_loss: 1.3495
Epoch 5/20
32/32 - 29s - 904ms/step - accuracy: 0.4667 - loss: 1.1074 - val_accuracy: 0.3383 - val_loss: 1.4561
Epoch 6/20
32/32 - 28s - 861ms/step - accuracy: 0.6398 - loss: 0.8475 - val_accuracy: 0.3542 - val_loss: 1.6346
Epoch 7/20
32/32 - 26s - 808ms/step - accuracy: 0.8756 - loss: 0.3842 - val_accuracy: 0.3443 - val_loss: 2.1488
Epoch 8/20
32/32 - 30s - 924ms/step - accuracy: 0.9582 - loss: 0.1315 - val_accuracy: 0.3393 - val_loss: 2.5580
Epoch 9/20
32/32 - 28s - 872ms/step - accuracy: 0.9751 - loss: 0.0707 - val_accuracy: 0.3502 - val_loss: 2.

KeyboardInterrupt: 